In [1]:
import pandas as pd
import os 
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import gaussian_kde

%matplotlib inline

In [2]:
! pwd

/cellar/users/mdow/Projects/HCC/RNA_analysis/DESeq_analysis


## Load input

In [3]:
indir='/cellar/users/mdow/Projects/HCC/RNA_analysis/'
myClustDf = pd.read_pickle('/cellar/users/mdow/Projects/HCC/RNA_analysis/color_map_human.pkl')
color_map_mouse = pd.read_pickle('/cellar/users/mdow/Projects/HCC/RNA_analysis/color_map_mouse.pkl')

pid_to_mGroups_dict = myClustDf.T['Mouse_def_groups'].to_dict()
pid_to_hoshida_dict = myClustDf.T['Hoshida'].to_dict()
pid_to_prolif_dict = myClustDf.T['Proliferative'].to_dict()


In [4]:
myClustDf

patient_id,a114,a4xi,a3m9,a3ma,a10x,a112,a113,a10w,a3mc,a3ch,...,a7mc,a7sf,a3a5,a2l6,a25z,a2kc,a10u,a217,a25s,a7il
iCluster,"(0.903114192626, 0.541853160718, 0.764951955571)","(0.903114192626, 0.541853160718, 0.764951955571)","(0.903114192626, 0.541853160718, 0.764951955571)","(0.903114192626, 0.541853160718, 0.764951955571)","(0.903114192626, 0.541853160718, 0.764951955571)","(0.903114192626, 0.541853160718, 0.764951955571)","(0.903114192626, 0.541853160718, 0.764951955571)","(0.903114192626, 0.541853160718, 0.764951955571)","(0.903114192626, 0.541853160718, 0.764951955571)","(0.903114192626, 0.541853160718, 0.764951955571)",...,"(0.998631295737, 0.85096502234, 0.184882741348)","(0.998631295737, 0.85096502234, 0.184882741348)","(0.998631295737, 0.85096502234, 0.184882741348)","(0.998631295737, 0.85096502234, 0.184882741348)","(0.998631295737, 0.85096502234, 0.184882741348)","(0.998631295737, 0.85096502234, 0.184882741348)","(0.998631295737, 0.85096502234, 0.184882741348)","(0.998631295737, 0.85096502234, 0.184882741348)","(0.998631295737, 0.85096502234, 0.184882741348)","(0.998631295737, 0.85096502234, 0.184882741348)"
Hoshida,grey,aqua,grey,grey,aqua,grey,grey,grey,grey,sienna,...,sienna,sienna,sienna,sienna,aqua,sienna,sienna,sienna,grey,sienna
Cirrhosis,lightgrey,lightgrey,lightgrey,lightgrey,lightgrey,lightgrey,lightgrey,lightgrey,lightgrey,lightgrey,...,lightgrey,lightgrey,lightgrey,lightgrey,lightgrey,black,lightgrey,lightgrey,lightgrey,lightgrey
HBV,lightgrey,lightgrey,lightgrey,lightgrey,lightgrey,lightgrey,lightgrey,black,lightgrey,black,...,black,lightgrey,lightgrey,lightgrey,black,lightgrey,lightgrey,lightgrey,lightgrey,lightgrey
HCV,black,lightgrey,lightgrey,lightgrey,lightgrey,lightgrey,lightgrey,lightgrey,lightgrey,lightgrey,...,lightgrey,black,lightgrey,lightgrey,lightgrey,black,lightgrey,lightgrey,lightgrey,lightgrey
Smoke,lightgrey,lightgrey,lightgrey,lightgrey,lightgrey,lightgrey,black,lightgrey,lightgrey,lightgrey,...,lightgrey,lightgrey,lightgrey,lightgrey,lightgrey,lightgrey,lightgrey,lightgrey,lightgrey,lightgrey
Alcohol,lightgrey,lightgrey,black,black,lightgrey,black,lightgrey,lightgrey,black,lightgrey,...,black,lightgrey,lightgrey,black,lightgrey,lightgrey,black,black,black,black
Gender,b,b,b,b,r,b,r,b,b,b,...,b,b,r,b,b,b,b,r,b,b
Grade,"(0.800984248694, 0.299146494094, 0.00839677095...","(0.800984248694, 0.299146494094, 0.00839677095...","(0.800984248694, 0.299146494094, 0.00839677095...","(0.996078431606, 0.60264516017, 0.163121881614)","(0.996078431606, 0.60264516017, 0.163121881614)","(0.996078431606, 0.60264516017, 0.163121881614)","(0.800984248694, 0.299146494094, 0.00839677095...","(0.800984248694, 0.299146494094, 0.00839677095...","(0.996078431606, 0.60264516017, 0.163121881614)","(0.996078431606, 0.60264516017, 0.163121881614)",...,"(0.800984248694, 0.299146494094, 0.00839677095...","(0.996078431606, 0.60264516017, 0.163121881614)","(0.996078431606, 0.60264516017, 0.163121881614)","(0.996078431606, 0.60264516017, 0.163121881614)","(0.996078431606, 0.60264516017, 0.163121881614)","(0.800984248694, 0.299146494094, 0.00839677095...","(0.996078431606, 0.60264516017, 0.163121881614)","(0.800984248694, 0.299146494094, 0.00839677095...","(0.996078431606, 0.60264516017, 0.163121881614)","(0.996170703804, 0.892041528926, 0.572595181185)"
Ethnicity,b,peru,peru,peru,yellowgreen,yellowgreen,yellowgreen,peru,peru,peru,...,yellowgreen,peru,yellowgreen,yellowgreen,peru,b,yellowgreen,yellowgreen,yellowgreen,peru


In [5]:
human_all = pd.read_csv(indir+'tcga_all_rawcounts.csv',index_col=0)

In [6]:
mouseClust = pd.read_csv(indir+'human_mouseClust.csv')

mouseClust.head()

,PID,myClust
0,a114,0
1,a4xi,1
2,a3m9,0
3,a3ma,0
4,a10x,1


## DESeq condition files 

In [6]:
#	MODEL	RNA_sname	condition
#J750_T1_S1	STAM	J750_T1_S1	Mut
#J750T2_S1	STAM	J750T2_S1	None
#J755_T1_S2	STAM	J755_T1_S2	Mut


In [7]:
human_all.shape

(9175, 236)

In [8]:
cond_base = pd.DataFrame(human_all.columns)
cond_base.columns = ['PID']

cond_base.index = cond_base.PID
cond_base.head()


,PID
PID,
4072,4072
4073,4073
5258,5258
5260,5260
5261,5261


In [9]:
hoshida_groups = {'aqua':'Hoshida_C1','grey':'Hoshida_C2','sienna':'Hoshida_C3'}

pid_to_hoshida_dict = myClustDf.T['Hoshida'].to_dict()

In [10]:
pid_to_mClust_dict = mouseClust.set_index(['PID'])['myClust'].to_dict()
mClust_groups = {0:'S1',1:'S2',2:'S3'}

## High proliferative vs. low proliferative

In [12]:
'''
Set proliferation colors
'''
iClust_prolif = pd.read_pickle('/cellar/users/mdow/Projects/HCC/RNA_analysis/iCluster/iclust_prolif_groups.pkl')

iClust_prolif.head()
sample_prolif_dict = iClust_prolif.set_index(['Sample'])['Group'].to_dict()


In [13]:
cond_base['condition'] = cond_base.PID.map(sample_prolif_dict)

prolif_cond = cond_base.dropna()

In [23]:
prolif_cond.to_csv('human/prolif_cond.csv')

In [34]:
prolif_cond.head()

,PID,condition
PID,,
4072,4072,high_prolif
4073,4073,high_prolif
5258,5258,high_prolif
5260,5260,high_prolif
5261,5261,high_prolif


In [30]:
human_iClust = human_all.loc[:,human_all.columns.isin(prolif_cond.PID.tolist())]

In [31]:
human_iClust.to_csv(indir+'tcga_iClust_rawcounts.csv')

In [32]:
human_iClust.head()

,4072,4073,5258,5260,5261,5262,5263,5264,a10r,a10s,...,a7mf,a7px,a7py,a7pz,a7s9,a7sb,a7sf,a7sk,a7xp,a82e
a1cf,0.0,0.0,9031.0,146.0,1296.0,1886.0,5404.0,8004.0,3642.0,2454.0,...,1355.0,688.0,2115.0,2901.0,1306.0,2958.0,2417.0,2531.0,4627.0,7.0
aaas,0.0,0.0,2054.0,1100.0,894.0,1203.0,1936.0,2677.0,1100.0,1048.0,...,2111.0,2271.0,844.0,4517.0,1381.0,1516.0,1070.0,1795.0,1176.0,1844.0
aacs,0.0,0.0,3308.0,1828.0,779.0,720.0,1713.0,1738.0,833.0,390.0,...,2158.0,2649.0,161.0,493.0,893.0,246.0,268.0,398.0,794.0,1767.0
aadac,0.0,0.0,5014.0,22.0,5842.0,16538.0,5372.0,2870.0,44042.0,13685.0,...,3210.0,909.0,4669.0,4501.0,6216.0,24838.0,5274.0,12267.0,18380.0,16.0
aadat,0.0,0.0,190.0,57.0,85.0,281.0,120.0,132.0,11.0,245.0,...,113.0,88.0,6.0,183.0,80.0,576.0,75.0,157.0,89.0,57.0


In [33]:
indir

'/cellar/users/mdow/Projects/HCC/RNA_analysis/'

## Generate separate condition files for each vs normal

### 1. Hoshida

In [63]:
cond_base['condition'] = cond_base.PID.map(pid_to_hoshida_dict).map(hoshida_groups).fillna('normal')

In [64]:
cond_base.condition.value_counts()

Hoshida_C2    83
Hoshida_C3    73
normal        53
Hoshida_C1    27
Name: condition, dtype: int64

In [66]:
hoshida_groups.values()

['Hoshida_C3', 'Hoshida_C1', 'Hoshida_C2']

In [73]:
for c in hoshida_groups.values():
    tmp = cond_base.copy()
    tmp.condition[tmp.condition != c] = 'normal'
    print tmp.condition.value_counts()
    
    fname = 'human/'+c+'_vs_normal.csv'
    tmp.to_csv(fname)
    

normal        163
Hoshida_C3     73
Name: condition, dtype: int64
normal        209
Hoshida_C1     27
Name: condition, dtype: int64
normal        153
Hoshida_C2     83
Name: condition, dtype: int64


In [106]:
fnames = sorted(pd.Series(os.listdir('human/'))[pd.Series(os.listdir('human/')).str.contains('_vs_normal')].tolist())
print ' '.join(fnames)

HCC_vs_normal.csv Hoshida_C1_vs_normal.csv Hoshida_C2_vs_normal.csv Hoshida_C3_vs_normal.csv S1_vs_normal.csv S2_vs_normal.csv S3_vs_normal.csv


In [110]:
fnames[0].split('.csv')[0]

'HCC_vs_normal'

In [111]:
print ' '.join([f.split('.csv')[0]+'_DESEq.csv' for f in fnames])

HCC_vs_normal_DESEq.csv Hoshida_C1_vs_normal_DESEq.csv Hoshida_C2_vs_normal_DESEq.csv Hoshida_C3_vs_normal_DESEq.csv S1_vs_normal_DESEq.csv S2_vs_normal_DESEq.csv S3_vs_normal_DESEq.csv


In [113]:
len(fnames)

7

### 2. my mouse clusters

In [85]:
cond_base['condition'] = cond_base.PID.map(pid_to_mClust_dict).map(mClust_groups).fillna('normal')

In [86]:
cond_base.condition.value_counts()

S1        87
S2        65
normal    53
S3        31
Name: condition, dtype: int64

In [87]:
for c in mClust_groups.values():
    tmp = cond_base.copy()
    tmp.condition[tmp.condition != c] = 'normal'
    print tmp.condition.value_counts()
    
    fname = 'human/'+c+'_vs_normal.csv'
    tmp.to_csv(fname)
    

normal    149
S1         87
Name: condition, dtype: int64
normal    171
S2         65
Name: condition, dtype: int64
normal    205
S3         31
Name: condition, dtype: int64


In [88]:
! ls human/

group1_vs_2and3.csv  group2_vs_3.csv	       Hoshida_C3_vs_normal.csv
group1_vs_2.csv      group3_vs_1and2.csv       S1_vs_normal.csv
group1_vs_3.csv      Hoshida_C1_vs_normal.csv  S2_vs_normal.csv
group2_vs_1and3.csv  Hoshida_C2_vs_normal.csv  S3_vs_normal.csv


### 3. All HCC vs normal

In [93]:
hcc_groups = {True:'normal',False:'tumor'}

In [94]:
cond_base['condition'] = cond_base.PID.str.contains('_n').map(hcc_groups)

In [96]:
cond_base.to_csv('human/HCC_vs_normal.csv')

### 4. TCGA 3 clusters

In [114]:
tcga_clustMems = pd.read_csv('/cellar/users/mdow/Projects/HCC/RNA_analysis/tcga_clust_mem_firebrowse.txt',sep='\t')

tcga_clustMems.columns = [u'SampleName', u'clu2', u'clu3', u'clu4', u'clu5', u'clu6',
       u'clu7', u'clu8', u'clu9', u'clu10']

tcga_clustMems['PID'] = [p.lower() for p in tcga_clustMems.SampleName.str.split('-').str[2]]

In [116]:
#tcga_clust3_cdict = dict(zip(tcga_clustMems.clu3.unique(), sns.color_palette('Set1',3)))
tcga_clust3_dict = tcga_clustMems.set_index('PID')['clu3'].to_dict()

In [120]:
tcga_clust3_groups = {1:'Tcga_c1',2:'Tcga_c2',3:'Tcga_c3'}

In [121]:
cond_base['condition'] = cond_base.PID.map(tcga_clust3_dict).map(tcga_clust3_groups).fillna('normal')

In [122]:
cond_base.condition.value_counts()

Tcga_c2    83
Tcga_c1    70
normal     53
Tcga_c3    30
Name: condition, dtype: int64

In [123]:
for c in tcga_clust3_groups.values():
    tmp = cond_base.copy()
    tmp.condition[tmp.condition != c] = 'normal'
    print tmp.condition.value_counts()
    
    fname = 'human/'+c+'_vs_normal.csv'
    tmp.to_csv(fname)
    

normal     166
Tcga_c1     70
Name: condition, dtype: int64
normal     153
Tcga_c2     83
Name: condition, dtype: int64
normal     206
Tcga_c3     30
Name: condition, dtype: int64


In [128]:
tmp = pd.Series(os.listdir('human/'))
print ' '.join(sorted(tmp[tmp.str.contains('Tcga')]))

Tcga_c1_vs_normal.csv Tcga_c2_vs_normal.csv Tcga_c3_vs_normal.csv
